# Developing a function to find transcription factor binding sites from differentially expressed genes

## Function aim
Build a command-line python program count the number of transcription factor binding sites (TFBs) from differentially expressed genes

## What are transcription factors
Transcription factors are proteins that regulate the transcription of genes—that is, their copying into RNA, on the way to making a protein.

https://www.khanacademy.org/science/biology/gene-regulation/gene-regulation-in-eukaryotes/a/eukaryotic-transcription-factors

## How do transcription factors work?

A typical transcription factor binds to DNA at a certain target sequence. Once it's bound, the transcription factor makes it either harder or easier for RNA polymerase to bind to the promoter of the gene.

Some transcription factors **activate transcription**. For instance, they may help the general transcription factors and/or RNA polymerase bind to the promoter, as shown in the diagram below.


In [4]:
# @hidden_cell
from IPython.display import display, Image
Image(url = 'https://ka-perseus-images.s3.amazonaws.com/6567f50d30ad3ac65aff1e815caf202b3abd7111.png')

Other transcription factors **repress transcription**. This repression can work in a variety of ways. As one example, a repressor may get in the way of the basal transcription factors or RNA polymerase, making it so they can't bind to the promoter or begin transcription.

## Binding sites

A typical transcription factor binds to DNA at a certain target sequence (or motif). Once it's bound, the transcription factor makes it either harder or easier for RNA polymerase to bind to the promoter of the gene, and consequently regulates the amount of messenger RNA (mRNA) produced by the gene. Some transcription factors activate transcription, while others repress transcription.
 
Transcription factor binding sites (TFBS) are often located in the 5’-upstream region of target genes to modulate the rate of gene transcription. DNA binding sites can be thus defined as short DNA sequences (typically 4 to 30 base pairs long) that are specifically bound by one or more DNA-binding proteins or protein complexes.

In [5]:
# @hidden_cell
Image(url = 'https://ka-perseus-images.s3.amazonaws.com/1ba8fe2b28b3dd5cd79ec75b74982ee87692dc9e.png')

The flexibility of DNA is what allows transcription factors at distant binding sites to do their job. The DNA loops like cooked spaghetti to bring far-off binding sites and transcription factors close to general transcription factors or "mediator" proteins.

In the cartoon above, an activating transcription factor bound at a far-away site helps RNA polymerase bind to the promoter and start transcribing.

To find and count TFBS my function should complete the following tasks:
1. Find the coordinates of differentially expressed genes or target genes
2. Extract the upstream sequences of these target genes
3. Find and count transcription factor binding sites (TFBSs) in the extracted sequences

### 0. Filter differentially expressed genes by logFC values

In [60]:
genes = ('/Volumes/sam079/RNAseq-POMV/Results/ControlvsPOMV6_ALL.csv')

In [61]:
import pandas as pd
import numpy as np
import math

In [136]:
genes = pd.read_csv('/Volumes/sam079/RNAseq-POMV/Results/ControlvsPOMV6_ALL.csv') # might crash in the office
genes = genes.dropna()

In [137]:
type(genes)

pandas.core.frame.DataFrame

In [148]:
genes = genes.astype({'ENTREZID':int}) 

In [149]:
genes.head()

,Row.names,logFC,AveExpr,t,P.Value,adj.P.Val,B,ENTREZID,GENENAME
0,4ebp,-0.813079,4.684809,-11.045991,4.993610e-11,2.048807e-09,14.885980,100194635,eukaryotic translation initiation factor 4E bi...
1,4ebp,-0.813079,4.684809,-11.045991,4.993610e-11,2.048807e-09,14.885980,100196411,Eukaryotic translation initiation factor 4E-1A...
2,4ebp,-0.813079,4.684809,-11.045991,4.993610e-11,2.048807e-09,14.885980,100194634,eukaryotic translation initiation factor 4E bi...
3,4f2,-0.171963,4.758786,-3.090667,4.904979e-03,1.800373e-02,-3.698283,100195278,4F2 cell-surface antigen heavy chain
4,a1cf,1.383839,-2.246948,2.553969,1.723450e-02,4.996076e-02,-3.093445,106589396,APOBEC1 complementation factor


In [150]:
DEgenes_up = genes.loc[genes['logFC'] >= 2]

In [151]:
DEgenes_up = DEgenes_up.sort_values(by = ['logFC'], ascending = False)
DEgenes_up

,Row.names,logFC,AveExpr,t,P.Value,adj.P.Val,B,ENTREZID,GENENAME
4488,LOC106578964,6.200508,2.541149,12.794806,2.247952e-12,1.355292e-10,15.519206,106578964,interferon-induced protein with tetratricopept...
10428,scyb7,6.176800,3.710864,5.567244,9.120766e-06,8.683798e-05,3.413170,100196194,Platelet basic protein
7066,LOC106595751,6.127157,3.705124,12.718014,2.558673e-12,1.513738e-10,16.973097,106595751,zinc-binding protein A33-like
3112,LOC106570889,5.823506,2.223820,4.572694,1.165131e-04,7.806751e-04,1.043902,106570889,C-X-C motif chemokine 11-like
1897,LOC106564387,5.717101,0.482356,7.924517,3.110612e-08,6.017787e-07,7.265915,106564387,uncharacterized LOC106564387
374,cmpk2,5.594479,5.914865,23.330002,2.640453e-18,7.869545e-16,30.961771,100136513,cytidine/uridine monophosphate kinase 2
7667,LOC106602323,5.540041,3.142172,10.419525,1.647317e-10,5.801789e-09,13.450069,106602323,probable E3 ubiquitin-protein ligase HERC3
1078,LOC100136541,5.452721,6.204966,17.167839,3.282131e-15,3.959268e-13,24.268506,100136541,ISG15-like protein
1339,LOC106561321,5.247778,-0.364280,8.534446,8.014250e-09,1.841354e-07,8.138895,106561321,uncharacterized LOC106561321
851,helz2,5.206373,6.323273,24.930777,5.492743e-19,2.041491e-16,33.177351,100380724,helicase with zinc finger 2


In [155]:
DEgenes_up = DEgenes_up[['ENTREZID']]

In [156]:
DEgenes_up.head()

,ENTREZID
4488,106578964
10428,100196194
7066,106595751
3112,106570889
1897,106564387


In [144]:
type(DEgenes_up)

pandas.core.series.Series

### 1. Find the coordinates of differentially expressed genes or target genes

https://www.toptal.com/python/comprehensive-introduction-your-genome-scipy

In [127]:
col_names = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']

In [168]:
df = pd.read_csv('/Users/sam079/Documents/2018_Transcription_factors/Data/GCF_000233375.1_ICSASG_v2_genomic.gff', 
                         sep='\t', comment='#', low_memory=False,
                         header=None, names=col_names)

In [169]:
df.seqid.unique()

array(['NC_027300.1', 'NC_027301.1', 'NC_027302.1', ..., 'NW_012498597.1',
       'NW_012561447.1', 'NC_001960.1'], dtype=object)

In [170]:
df.type.value_counts()

exon                     1313909
CDS                      1155790
region                    232155
mRNA                       97746
gene                       79030
cDNA_match                 41529
tRNA                       23676
lnc_RNA                     9435
transcript                  2603
pseudogene                  2556
V_gene_segment               144
C_gene_segment                38
rRNA                           2
D_loop                         1
origin_of_replication          1
Name: type, dtype: int64

In [171]:
bestref = df[df.source.isin(['BestRefSeq'])]
bestref.sample(20)

,seqid,source,type,start,end,score,strand,phase,attributes
765926,NC_027308.1,BestRefSeq,CDS,17016238,17016423,.,-,2,ID=cds28891;Parent=rna33150;Dbxref=GeneID:1001...
820820,NC_027308.1,BestRefSeq,exon,66053376,66054575,.,-,.,ID=id398971;Parent=rna35258;Dbxref=GeneID:1001...
2170389,NC_027321.1,BestRefSeq,exon,51124260,51124558,.,-,.,ID=id1057465;Parent=rna87789;Dbxref=GeneID:100...
453264,NC_027303.1,BestRefSeq,CDS,48241592,48241753,.,-,2,ID=cds16500;Parent=rna18890;Dbxref=GeneID:1001...
2125571,NC_027321.1,BestRefSeq,exon,17220330,17220397,.,+,.,ID=id1035226;Parent=rna86112;Dbxref=GeneID:100...
2204442,NC_027322.1,BestRefSeq,CDS,22692679,22692743,.,+,2,ID=cds78518;Parent=rna89039;Dbxref=GeneID:1001...
2584578,NW_012341938.1,BestRefSeq,exon,10281,10378,.,-,.,ID=id1265106;Parent=rna104255;Dbxref=GeneID:10...
1499991,NC_027314.1,BestRefSeq,exon,17399415,17399543,.,+,.,ID=id729124;Parent=rna61976;Dbxref=GeneID:1001...
766023,NC_027308.1,BestRefSeq,exon,17068049,17068125,.,-,.,ID=id372042;Parent=rna33154;Dbxref=GeneID:1001...
1176020,NC_027311.1,BestRefSeq,CDS,42141299,42141371,.,+,0,ID=cds43438;Parent=rna49449;Dbxref=GeneID:1001...


In [172]:
CDS = bestref[bestref.type == 'CDS']

In [173]:
CDS = CDS.copy()

In [174]:
CDS.sample(10).attributes.values

array(['ID=cds52243;Parent=rna59369;Dbxref=GeneID:100380835,Genbank:NP_001167428.1;Name=NP_001167428.1;Note=The RefSeq protein aligns at 85%25 coverage compared to this genomic sequence;exception=annotated by transcript or proteomic data;gbkey=CDS;gene=orc2;inference=similar to AA sequence (same species):RefSeq:NP_001167428.1;partial=true;product=Origin recognition complex subunit 2;protein_id=NP_001167428.1',
       'ID=cds134;Parent=rna161;Dbxref=GeneID:100196112,Genbank:NP_001134613.1;Name=NP_001134613.1;Note=The RefSeq protein has 1 substitution compared to this genomic sequence;exception=annotated by transcript or proteomic data;gbkey=CDS;gene=cdkn3;inference=similar to AA sequence (same species):RefSeq:NP_001134613.1;product=cyclin-dependent kinase inhibitor 3;protein_id=NP_001134613.1',
       'ID=cds44351;Parent=rna50430;Dbxref=GeneID:100195178,Genbank:NP_001133679.1;Name=NP_001133679.1;gbkey=CDS;gene=rheb;product=ras homolog enriched in brain;protein_id=NP_001133679.1',
      

In [175]:
import re


RE_GENE_NAME = re.compile(r'GeneID:(?P<ENTREZID>[0-9]+)')

def extract_gene_name(attributes_str):
    res = RE_GENE_NAME.search(attributes_str)
    return res.group('ENTREZID')


CDS['ENTREZID'] = CDS.attributes.apply(extract_gene_name)

In [176]:
RE_DESC = re.compile('product=(?P<desc>.+?);')
def extract_description(attributes_str):
    res = RE_DESC.search(attributes_str)
    if res is None:
        return ''
    else:
        return res.group('desc')


CDS['desc'] = CDS.attributes.apply(extract_description)

In [177]:
CDS.drop('attributes', axis=1, inplace=True)
CDS

,seqid,source,type,start,end,score,strand,phase,ENTREZID,desc
497,NC_027300.1,BestRefSeq,CDS,1342527,1342649,.,+,0,100316613,interferon promoter stimulating protein 1
498,NC_027300.1,BestRefSeq,CDS,1342922,1343099,.,+,0,100316613,interferon promoter stimulating protein 1
499,NC_027300.1,BestRefSeq,CDS,1343567,1344243,.,+,2,100316613,interferon promoter stimulating protein 1
500,NC_027300.1,BestRefSeq,CDS,1345061,1345903,.,+,0,100316613,interferon promoter stimulating protein 1
2020,NC_027300.1,BestRefSeq,CDS,3499221,3499242,.,-,0,100194980,Mediator of RNA polymerase II transcription su...
2021,NC_027300.1,BestRefSeq,CDS,3498755,3498917,.,-,2,100194980,Mediator of RNA polymerase II transcription su...
2022,NC_027300.1,BestRefSeq,CDS,3498083,3498174,.,-,1,100194980,Mediator of RNA polymerase II transcription su...
2023,NC_027300.1,BestRefSeq,CDS,3496072,3496154,.,-,2,100194980,Mediator of RNA polymerase II transcription su...
2024,NC_027300.1,BestRefSeq,CDS,3495610,3495718,.,-,0,100194980,Mediator of RNA polymerase II transcription su...
2025,NC_027300.1,BestRefSeq,CDS,3494733,3494851,.,-,2,100194980,Mediator of RNA polymerase II transcription su...


In [237]:
from pyfaidx import Fasta

In [239]:
genome = Fasta('/Volumes/OSM_CBR_AF_POMV_work/POMV_RNA_seq/Genomes/Salmo_salar/GCF_000233375.1_ICSASG_v2_genomic.fna')

In [240]:
CDS.head()

,seqid,source,type,start,end,score,strand,phase,ENTREZID,desc
497,NC_027300.1,BestRefSeq,CDS,1342527,1342649,.,+,0,100316613,interferon promoter stimulating protein 1
498,NC_027300.1,BestRefSeq,CDS,1342922,1343099,.,+,0,100316613,interferon promoter stimulating protein 1
499,NC_027300.1,BestRefSeq,CDS,1343567,1344243,.,+,2,100316613,interferon promoter stimulating protein 1
500,NC_027300.1,BestRefSeq,CDS,1345061,1345903,.,+,0,100316613,interferon promoter stimulating protein 1
2020,NC_027300.1,BestRefSeq,CDS,3499221,3499242,.,-,0,100194980,Mediator of RNA polymerase II transcription su...


In [196]:
DEgenes_up = DEgenes_up.astype(str)

In [197]:
DEgenes_up.dtypes

ENTREZID    object
dtype: object

In [198]:
CDS.dtypes

seqid       object
source      object
type        object
start        int64
end          int64
score       object
strand      object
phase       object
ENTREZID    object
desc        object
dtype: object

In [200]:
newdf = pd.merge(DEgenes_up, CDS)

In [203]:
newdf_small = newdf.head()
newdf_small

,ENTREZID,seqid,source,type,start,end,score,strand,phase,desc
0,100196194,NC_027314.1,BestRefSeq,CDS,3515019,3515088,.,+,0,Platelet basic protein precursor
1,100196194,NC_027314.1,BestRefSeq,CDS,3515273,3515399,.,+,2,Platelet basic protein precursor
2,100196194,NC_027314.1,BestRefSeq,CDS,3515676,3515768,.,+,1,Platelet basic protein precursor
3,100196194,NC_027314.1,BestRefSeq,CDS,3516526,3516538,.,+,1,Platelet basic protein precursor
4,100136513,NC_027308.1,BestRefSeq,CDS,10605995,10606582,.,-,0,thymidylate kinase


In [299]:
mylist = []
mydict = {}
for index, row in newdf_small.iterrows():
            genes = row['ENTREZID']
            sequences = genome[row['seqid']][row['start'] - 3:row['start']].seq
            mydict[genes] = sequences
            mylist.append(mydict)
            mydict = {}
print(mylist)
            

[{'100196194': 'CCA'}, {'100196194': 'AGG'}, {'100196194': 'AGT'}, {'100196194': 'AGG'}, {'100136513': 'ACC'}]


In [300]:
mylist = []
mydict = {}
for index, row in newdf_small.iterrows():
            genes = row['ENTREZID']
            sequences = genome[row['seqid']][row['start']:row['start'] + 3].seq
            mydict[genes] = sequences
            mylist.append(mydict)
            mydict = {}
print(mylist)

[{'100196194': 'TGA'}, {'100196194': 'CCA'}, {'100196194': 'GTC'}, {'100196194': 'AGT'}, {'100136513': 'TGT'}]
